## Cross check maps

In [1]:
import pandas as pd
import numpy as np

### Naming scheme

- PolarFire: DPM 
  - Range: [0,1]
- HGCROC: Board 
  - 4 boards per DPM but only 3 connected 
  - Range: [0,1,2]
- Channel:
  - Channels for the lower half (0-35)
  - Channels for the upper half (36-71)
  - 36 channels per half of the HGCROC: 0-71 in total
  - Channels "inactive" in 0-71 range: 8,17,26,35,44,53,62,71
  - Range: [0 - 70]
- Elink:
  - Each HGCROC mapped has 2 elinks
  - Range: [0,1]
  - Range in pflib: 
    - [0,1,2,3,4,5] 
    - (0,1): ROC 0, (2,3): ROC 1, (4,5): ROC 2
- Bar: strip
  - Range: [1,2,3,4]
  - AllBar: If we group strips by layers:
    - Front: 
      2 CMBs: [1-8]
    - Back:
      3 CMBs: [1-12]
- Quadbar:
  - Quarbar groups 4 strips
  - Range: 
    - [1,2]: 2CMBs in the front (layers 0,9)
    - [1,2,3]: 3CMBs in the back (layers 10,19)
- CMB:
  - Each quadbar is connected to 2 CMBs, one in each End
  - In total each board is connected to 16 CMBs.
  - Range: [1-16]
- Plane: Layer
  - 19 Layers
  - 9 layers in front: 4 vertical, 5 horizontal
  - 10 layers in back: 5 vertical, 5 horizontal
  - Range: [1-19]
- Cable:
  - Cable numbering
  - Range: [11-196]
- Orientation:
  - Vertical or Horizontal
- Side: End
  - Up/Down or Left/Right
  - In ldmx-sw: Positive end (end=0), corresponds to Top,Left. Negative end (end=1), corresponds to Bottom,Right.
  - Range: [U/L,D/R]
  - Range: [0,1]

First, this is the map from Matt (and David) in ldmx-sw.

In [2]:
ldmxsw_map = pd.read_csv("testbeam_connections_updated.csv") # tb file
ldmxsw_map

,Polarfire,HGCROC,Channel,CMB,Quadbar,Bar,Plane,Cable
0,0,0,0,1,1,4,1,11
1,0,0,1,1,1,3,1,11
2,0,0,2,1,1,2,1,11
3,0,0,3,1,1,1,1,11
4,0,0,4,2,1,1,1,12
...,...,...,...,...,...,...,...,...
379,1,2,66,15,2,1,19,195
380,1,2,67,16,2,1,19,196
381,1,2,68,16,2,2,19,196
382,1,2,69,16,2,3,19,196


Now, this is the map provided by Lennart.

In [3]:
tmap = pd.read_csv("testbeam_connections_Lennart.csv") # provided by Lennart
tmap

,#Dpm,Board,Elink,Connector,Cable,Plane,Orientation,QuadBar,Side,Rotated
0,0,0,0,J1,11,0,V,0,U,NaN
1,0,0,0,J2,12,0,V,0,D,NaN
2,0,0,0,J3,13,0,V,1,U,NaN
3,0,0,0,J4,14,0,V,1,D,NaN
4,0,0,0,J5,21,1,H,0,L,Yes
...,...,...,...,...,...,...,...,...,...,...
91,1,2,1,J12,192,18,V,0,D,NaN
92,1,2,1,J13,193,18,V,1,U,NaN
93,1,2,1,J14,194,18,V,1,D,NaN
94,1,2,1,J15,195,18,V,2,U,NaN


Need to convert Lennart's format into Tom's/Matt's.
First, let's rename and re-arrange some columns to have the same name as range as in the ldmxsw file.

In [4]:
tb_map = pd.read_csv("testbeam_connections_Lennart.csv") # provided by Lennart
tb_map = tb_map.loc[tb_map.index.repeat(4)] # need to repeat rows by 4 (since we have 4 bars per connector)
tb_map.rename(columns={'Board': 'HGCROC', 'QuadBar': 'Quadbar', '#Dpm': 'Polarfire', 'Connector': 'CMB', 'Side': 'End'}, inplace=True)
tb_map

,Polarfire,HGCROC,Elink,CMB,Cable,Plane,Orientation,Quadbar,End,Rotated
0,0,0,0,J1,11,0,V,0,U,NaN
0,0,0,0,J1,11,0,V,0,U,NaN
0,0,0,0,J1,11,0,V,0,U,NaN
0,0,0,0,J1,11,0,V,0,U,NaN
1,0,0,0,J2,12,0,V,0,D,NaN
...,...,...,...,...,...,...,...,...,...,...
94,1,2,1,J15,195,18,V,2,U,NaN
95,1,2,1,J16,196,18,V,2,D,NaN
95,1,2,1,J16,196,18,V,2,D,NaN
95,1,2,1,J16,196,18,V,2,D,NaN


In [5]:
connector_map = dict(zip([f'J{j+1}' for j in range(16)], list(range(1,17)))) # replace connector strings
tb_map.replace({'CMB': connector_map}, inplace=True)

tb_map.fillna(False,inplace=True)  # replace Rotated by False/True
tb_map.loc[tb_map['Rotated'] =='Yes', 'Rotated'] = True

tb_map.set_index(['Cable'],inplace=True) # sort by cable
tb_map.sort_index(inplace=True)
tb_map.reset_index(inplace=True)

# get 2 elinks per HGCROC (0,1) to (0,5)
tb_map['Elink'] = tb_map['Elink']+2*tb_map['HGCROC']

# get plane numbering 1-19 instead of 0-18
tb_map['Plane'] += 1

# get quadbar numbering 0-2 to 1-3
tb_map['Quadbar'] += 1

# Ends convention in ldmx-sw
# Positive end (end=0), corresponds to Top,Left.
# Negative end (end=1), corresponds to Bottom,Right.
tb_map.loc[tb_map['End'] =='U', 'End'] = 0
tb_map.loc[tb_map['End'] =='D', 'End'] = 1
tb_map.loc[tb_map['End'] =='L', 'End'] = 0
tb_map.loc[tb_map['End'] =='R', 'End'] = 1

tb_map

,Cable,Polarfire,HGCROC,Elink,CMB,Plane,Orientation,Quadbar,End,Rotated
0,11,0,0,0,1,1,V,1,0,False
1,11,0,0,0,1,1,V,1,0,False
2,11,0,0,0,1,1,V,1,0,False
3,11,0,0,0,1,1,V,1,0,False
4,12,0,0,0,2,1,V,1,1,False
...,...,...,...,...,...,...,...,...,...,...
379,195,1,2,5,15,19,V,3,0,False
380,196,1,2,5,16,19,V,3,1,False
381,196,1,2,5,16,19,V,3,1,False
382,196,1,2,5,16,19,V,3,1,False


Now we can add the channel column as well. 
This should be identical to the channels in ldmxsw.
We remove every 8th row.
Should we remove the calibration channels as well?

In [6]:
channels_ldmxsw = list(ldmxsw_map['Channel'].unique())
channels = list(range(72))
channels.remove(8)
channels.remove(17) 
channels.remove(26)
channels.remove(35)
channels.remove(44)
channels.remove(53)
channels.remove(62)
channels.remove(71)
tb_map['Channel'] = pd.Series(channels*tb_map.shape[0])
print(channels_ldmxsw==channels)

True


Now that we have the table ordered by Cable, let's introduce the Bar column.
Ranges from 1-4 but we need to re-order the rows where `Rotated==True`.

In [7]:
tb_map['Bar'] = pd.Series(list(range(1,5))*tb_map.shape[0])

new_map = tb_map.copy()
new_map.loc[(tb_map['Rotated'] == True) & (tb_map['Bar']==1), 'Bar'] = 4
new_map.loc[(tb_map['Rotated'] == True) & (tb_map['Bar']==2), 'Bar'] = 3
new_map.loc[(tb_map['Rotated'] == True) & (tb_map['Bar']==3), 'Bar'] = 2
new_map.loc[(tb_map['Rotated'] == True) & (tb_map['Bar']==4), 'Bar'] = 1
new_map

,Cable,Polarfire,HGCROC,Elink,CMB,Plane,Orientation,Quadbar,End,Rotated,Channel,Bar
0,11,0,0,0,1,1,V,1,0,False,0,1
1,11,0,0,0,1,1,V,1,0,False,1,2
2,11,0,0,0,1,1,V,1,0,False,2,3
3,11,0,0,0,1,1,V,1,0,False,3,4
4,12,0,0,0,2,1,V,1,1,False,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
379,195,1,2,5,15,19,V,3,0,False,66,4
380,196,1,2,5,16,19,V,3,1,False,67,1
381,196,1,2,5,16,19,V,3,1,False,68,2
382,196,1,2,5,16,19,V,3,1,False,69,3


In [8]:
# let's also introduce Bars that range from 1-12
new_map['AllBar'] = tb_map.apply(lambda row: row.Bar+4*(row.Quadbar-1), axis=1)
new_map

,Cable,Polarfire,HGCROC,Elink,CMB,Plane,Orientation,Quadbar,End,Rotated,Channel,Bar,AllBar
0,11,0,0,0,1,1,V,1,0,False,0,1,1
1,11,0,0,0,1,1,V,1,0,False,1,2,2
2,11,0,0,0,1,1,V,1,0,False,2,3,3
3,11,0,0,0,1,1,V,1,0,False,3,4,4
4,12,0,0,0,2,1,V,1,1,False,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,195,1,2,5,15,19,V,3,0,False,66,4,12
380,196,1,2,5,16,19,V,3,1,False,67,1,9
381,196,1,2,5,16,19,V,3,1,False,68,2,10
382,196,1,2,5,16,19,V,3,1,False,69,3,11


Let's save a long map now with the PolarFile, Elink, Orientation and Side.

In [9]:
new_long_map = new_map[['HGCROC','Channel','CMB','Quadbar','Bar','Plane','AllBar','Elink','End']]
new_long_map

,HGCROC,Channel,CMB,Quadbar,Bar,Plane,AllBar,Elink,End
0,0,0,1,1,1,1,1,0,0
1,0,1,1,1,2,1,2,0,0
2,0,2,1,1,3,1,3,0,0
3,0,3,1,1,4,1,4,0,0
4,0,4,2,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...
379,2,66,15,3,4,19,12,5,0
380,2,67,16,3,1,19,9,5,1
381,2,68,16,3,2,19,10,5,1
382,2,69,16,3,3,19,11,5,1


In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(new_long_map)

     HGCROC  Channel  CMB  Quadbar  Bar  Plane  AllBar  Elink End
0         0        0    1        1    1      1       1      0   0
1         0        1    1        1    2      1       2      0   0
2         0        2    1        1    3      1       3      0   0
3         0        3    1        1    4      1       4      0   0
4         0        4    2        1    1      1       1      0   1
5         0        5    2        1    2      1       2      0   1
6         0        6    2        1    3      1       3      0   1
7         0        7    2        1    4      1       4      0   1
8         0        9    3        2    1      1       5      0   0
9         0       10    3        2    2      1       6      0   0
10        0       11    3        2    3      1       7      0   0
11        0       12    3        2    4      1       8      0   0
12        0       13    4        2    1      1       5      0   1
13        0       14    4        2    2      1       6      0   1
14        

In [11]:
new_map = new_map[['Polarfire','HGCROC','Channel','CMB','Quadbar','Bar','Plane','Cable']]
new_map

,Polarfire,HGCROC,Channel,CMB,Quadbar,Bar,Plane,Cable
0,0,0,0,1,1,1,1,11
1,0,0,1,1,1,2,1,11
2,0,0,2,1,1,3,1,11
3,0,0,3,1,1,4,1,11
4,0,0,4,2,1,1,1,12
...,...,...,...,...,...,...,...,...
379,1,2,66,15,3,4,19,195
380,1,2,67,16,3,1,19,196
381,1,2,68,16,3,2,19,196
382,1,2,69,16,3,3,19,196


In [12]:
ne_stacked = (ldmxsw_map != new_map).stack()
changed = ne_stacked[ne_stacked]
    
difference_locations = np.where(ldmxsw_map != new_map)
changed_from = ldmxsw_map.values[difference_locations]
changed_to = new_map.values[difference_locations]
pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)

,,from,to
0,Bar,4,1
1,Bar,3,2
2,Bar,2,3
3,Bar,1,4
8,Bar,4,1
...,...,...,...
379,Bar,1,4
380,Quadbar,2,3
381,Quadbar,2,3
382,Quadbar,2,3


Now, let's cross check Peter's map.

In [13]:
# converts the link-channel provided by the HGCROC into a 'real' channel: 
# goes from 1 to 384, representing a SiPM each
# link is the chip halves, channel is just the channel
def FpgaLinkChannel_to_realChannel(FpgaLinkChannel):                                                                                                   
    #channel = FpgaLinkChannel[2] # Should this be -1?
    channel = FpgaLinkChannel[2]-1
    if 0 <= channel and channel <= 7:  realChannel = channel
    elif 9 <= channel and channel <= 16:  realChannel = channel-1
    elif 19 <= channel and channel <= 26:  realChannel = channel-3
    elif 28 <= channel and channel <= 35:  realChannel = channel-4
    else: realChannel = None

    if realChannel != None: realChannel+=FpgaLinkChannel[1]*32
    if realChannel != None: realChannel+=FpgaLinkChannel[0]*32*6
    return realChannel

#converts the 'real' channel into a 3 vector that describes the SiPM really well   
# take realChannel return [layer,bar,side]
def realChannel_to_SipM(c):                                                                                                                                                           
    if c == None: return None
    for i in range(1,10):
        if 0 <= c and c <= 3: return [i,c,0]
        if 4 <= c and c <= 7: return [i,c-4,1]
        if 8 <= c and c <= 11: return [i,c-4,0]
        if 12 <= c and c <= 15: return [i,c-8,1]
        c-=16
    for i in range(10,20):
        if 0 <= c and c <= 3: return [i,c,0]
        if 4 <= c and c <= 7: return [i,c-4,1]
        if 8 <= c and c <= 11: return [i,c-4,0]
        if 12 <= c and c <= 15: return [i,c-8,1]
        if 16 <= c and c <= 19: return [i,c-8,0]
        if 20 <= c and c <= 23: return [i,c-12,1]
        c-=24
    return 'too many layers'

Let's test Peter's mapping with FPGA=1, link=1, channel=1

In [14]:
# call as: FpgaLinkChannel_to_realChannel(fpga,link,channel)
realChannel = FpgaLinkChannel_to_realChannel([0,1,1])
print(realChannel)
#PlaneBarEnd = realChannel_to_SipM(realChannel)
LayerBarSide = realChannel_to_SipM(realChannel)
print(LayerBarSide)

32
[3, 0, 0]


In [15]:
new_long_map.query('(Plane==13) & (Bar==4) & (End==0) & (Quadbar==2)')

,HGCROC,Channel,CMB,Quadbar,Bar,Plane,AllBar,Elink,End
227,0,39,9,2,4,13,8,1,0


In [16]:
new_long_map.query('(HGCROC==0) & (Elink==0)')

,HGCROC,Channel,CMB,Quadbar,Bar,Plane,AllBar,Elink,End
0,0,0,1,1,1,1,1,0,0
1,0,1,1,1,2,1,2,0,0
2,0,2,1,1,3,1,3,0,0
3,0,3,1,1,4,1,4,0,0
4,0,4,2,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...
219,0,30,7,1,4,13,4,0,0
220,0,31,8,1,1,13,1,0,1
221,0,32,8,1,2,13,2,0,1
222,0,33,8,1,3,13,3,0,1


In [17]:
new_long_map.to_csv("testbeam_connections_Apr13.csv")